In [4]:
import pandas as pd
import numpy as np
import re
import os

In [5]:
# Set the path to the data folder
data_folder = os.path.join('..', 'data')  # Assuming your current working directory is 'bot'

# Set the path to the text file
text_file_path = os.path.join(data_folder, '1.txt')

# Read the content of the text file
with open(text_file_path, 'r') as file:
    text_data = file.read()

# Now 'text_data' contains the content of the file
# You can use 'text_data' for training your chatbot
print(text_data)

[q] What is GPA?
[a] The GPA, or Grade Point Average, is a number that indicates how high you scored in your courses on average. Using a scale from 1.0 to 4.0, your GPA tracks your progress during your studies. This number is used to assess whether you meet the standards and expectations set by the degree program or university.

[q] What is a good GPA?
[a] A good GPA on a standard 4.0 scale can fall between 3.0 and 4.0â€”or between a B and an A+.

[q] How to calculate GPA?
[a] Your GPA will be calculated using the following formula and rounded to the second decimal.GPA= âˆ‘(Grade points scored for the couese unit * Credit value of the course unit)/(Cumulative credit value of all course units followed) 

[q] At which year are we getting selected for a special degree?
[a] There is no special degree for ICT students. CS special students are selected by the end of the second year, second-semester examination.

[q] What are considered for a special degree in CS?
[a] Your overall GPA at the 

In [6]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # # Remove punctuation
    # text = re.sub(r'[^\w\s]', '', text)

    # Standardize contractions
    text = re.sub(r"can't", "cannot", text)

    # # Remove extra whitespaces
    # text = re.sub(r'\s+', ' ', text).strip()

    # # Remove excess newline characters
    # text = re.sub(r'\n+', '\n', text).strip()

    return text

In [7]:
text_data = preprocess_text(text_data)

In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

d:\research\implementation\research-academic-counselling-chatbot\.torch_env_3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [10]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator


In [11]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)
  print(train_dataset)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [12]:
# Model and Training Parameters
train_file_path = "/content/drive/MyDrive/Colab Notebooks/data/train.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/Colab Notebooks/models/chat_models/model1'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 10.0
save_steps = 50000

In [13]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

ImportError: 
TextDataset requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFTextDataset".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model, tokenizer, sequence, max_length):
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))